In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from lesson_functions import *
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
#from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib
import gc, pickle
%matplotlib

Using matplotlib backend: Qt4Agg


/home/batbold/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
def preprocess_image(cars, notcars, color_space='YCrCb', hog_channel = 'ALL',  
                     spatial_size = (32, 32), hist_bins = 32, orient = 9, 
                     pix_per_cell = 8, cell_per_block = 2 ):
    ### TODO: Tweak these parameters and see how the results change.
    
    spatial_feat = True # Spatial features on or off
    hist_feat = True # Histogram features on or off
    hog_feat = True # HOG features on or off
    y_start_stop = [None, None] # Min and max in y to search in slide_window()

    car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
    notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

    X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    X = X_scaler.transform(X)

    # Define the labels vector
    y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
    return X,y,X_scaler

In [ ]:
# Read in car and non-car images
images = glob.glob('../*vehicles/*/*.png')

cars = []
notcars = []
for image in images:
    if 'non-vehicles' in image:
        notcars.append(image)
    else:
        cars.append(image)

In [ ]:
print(len(cars),len(notcars))

In [ ]:
color_space='LUV'
hog_channel = 0
spatial_size = (32, 32)
hist_bins = 32
orient = 9
pix_per_cell = 8
cell_per_block = 2

scaled_X,y,X_scaler=preprocess_image(cars, notcars, color_space=color_space, hog_channel = hog_channel,  
                     spatial_size = spatial_size, hist_bins = hist_bins, orient = orient, 
                     pix_per_cell = pix_per_cell, cell_per_block = cell_per_block)

print(len(scaled_X))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2,random_state=42)

print('Feature vector length:', len(X_train))

In [ ]:
# Use a linear SVC 
svc = LinearSVC()
#svc = SVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()

In [ ]:
print( color_space, orient, pix_per_cell, cell_per_block, hog_channel, spatial_size, hist_bins)

In [ ]:

data={
    'svc': svc,
    'X_scaler': X_scaler,
    'color_space': color_space,
    'orient': orient,
    'pix_per_cell': pix_per_cell,
    'cell_per_block': cell_per_block,
    'spatial_size' : spatial_size,
    'hist_bins': hist_bins,
    'hog_channel': hog_channel
     }

with open('svc_params_LUV_hog0_orient9_size32_pix8_cell2.pk', 'wb') as pFile:
    pickle.dump(data, pFile)   